In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import Utils.model_setup
import torch

model = torch.load("denoise_model_complete.pth", map_location=torch.device("cpu"))
model.eval()

/var/folders/9n/xs76t_gx4knbv1_tl_d8gp940000gn/T/ipykernel_32907/588429505.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("denoise_model_complete.pth

DenoiseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1032, out_features=1024, bias=True)
  (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=1024, out_features=129, bias=True)
)

In [10]:
import torchaudio
import torch
from pathlib import Path

window_size = 256
window_length= window_size
overlap= int(0.75 * window_size)
fft_length= window_size
num_features= window_size // 2 + 1
num_segments= 8
batch_size= 128
epochs= 3
lr= 1e-5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
window = torch.hamming_window(window_length, periodic=True)


def denoise_audio(model, noisy_waveform):
    model.eval()
    noisy_waveform = noisy_waveform.to(device)

    # STFT del audio ruidoso
    noisy_stft = torch.stft(noisy_waveform.squeeze(), n_fft=fft_length,
                            hop_length=window_length - overlap,
                            window=window, return_complex=True)
    noisy_mag = noisy_stft.abs()
    noisy_phase = noisy_stft.angle()


    # Segmentar magnitudes
    padded = torch.cat([noisy_mag[:, :num_segments-1], noisy_mag], dim=1)
    segments = []
    for i in range(noisy_mag.shape[1]):
        seg = padded[:, i:i+num_segments]
        segments.append(seg.unsqueeze(0))
    X_proc = torch.stack(segments, dim=0).unsqueeze(1).to(device)

    with torch.no_grad():
        preds = model(X_proc)
    preds = preds.cpu()

    clean_mag_est = preds.T

    clean_stft_est = torch.polar(clean_mag_est, noisy_phase)


    clean_waveform = torch.istft(clean_stft_est, n_fft=fft_length,
                                 hop_length=window_length - overlap,
                                 window=window.cpu())

    return clean_waveform

noisy_folder = "../Audios/Augmented Audios 3 SNR"
noisy_audios = list(Path(noisy_folder).rglob('*.flac'))
noisy_input = torchaudio.load(noisy_audios[3])

denoised_output = denoise_audio(model, noisy_input[0])

# Guardar resultados
torchaudio.save("noisy.wav", noisy_input[0].cpu(), 8000)
torchaudio.save("denoised.wav", denoised_output.unsqueeze(0).cpu(), 8000)

print("Audios guardados: noisy.wav y denoised.wav")

Audios guardados: noisy.wav y denoised.wav
